# Generating Customer Event Data 
## Imports

In [360]:
from random import uniform, choice, choices, gauss

## Conversion Rates
Found [this](https://www.quora.com/How-many-times-does-a-customer-visit-a-website-before-making-a-purchase)
interesting data about the number of visits that users typically have before a conversion on an 
ecommerce site. 
* 73% of conversions occur on the first visit
* 14% of conversions occur on the second visit
* 5% of conversions occur on the third visit
* The average conversion rate for these sites was 1.9%

I also found [this](https://www.statista.com/topics/871/online-shopping/) site listing the average 
conversion rate across all verticals at 2.72\% for 2018 - 2019. We'll use that as our conversion rate
for now.

In [361]:
CONVERSION_RATE = 0.0272

def touches_before_conversion(num_to_generate=1):
    yield choices([1, 2, 3], [73, 14, 5], k=num_to_generate)

def is_conversion():
    return uniform(0, 1) <= CONVERSION_RATE

## Time distribution within an event chain
We typically use Poisson distributions to model events within an event chain. Not sure that we're 
going to need one in this case, but I make a note of it here. 
We'll have to install numpy to get that working, it's not part of the standard library. 

I found [this](https://neilpatel.com/blog/8-important-ecommerce-stats/) blog post talking about some
time statistics on websites. Looking at point 5 we can summarize these numbers:
* New users 
    * Average of 151 s on the site
    * View an average of 3.88 pages
* Old Users
    * Average 331 s on the site
    * View an average of 5.55 pages 

We'll say that we have a 70/30 split of new to return customers. We'll update that when / if we get 
some additional information around this. We'll also assume a 10% error on the measurements

In [362]:
def is_new_customer():
    return uniform(0, 1) <= 0.70
    
def generate_visit_stats(is_new):
    if is_new:
        return gauss(151, 15.1), gauss(3.88, 0.388)
    return gauss(331, 33.1), gauss(5.55, 0.555)
    

## Types of event chains
We'll have several different event chains that we will want to model.

| Type         | Chain                                |
|--------------|--------------------------------------|
| Search       | Search -> Page View                  |
| View         | N Page Views                         |
| Conversion   | N Page Views -> Cart Add -> Checkout |
| Cart Abandon | N Page Views -> Cart Add -> No Checkout within X days |

These should cover most scenarios for now. We will need to ensure that these follow a logical 
time flow. 

In [363]:
EVENT_CHAINS = {
    'search': {'chain': ['search', 'view'], 'weight': 0.8},
    'view': {'chain': ['view'], 'weight': 0.11},
    'direct_abandon': {'chain': ['view', 'add'], 'weight': 0.03},
    'search_abandon': {'chain': ['search', 'view', 'add'], 'weight': 0.03},
    'delete_abandon': {'chain': ['view', 'add', 'remove'], 'weight': 0.03},
    'direct_conversion': {'chain': ['view', 'add', 'purchase'], 'weight': 0.2},
    'search_conversion': {'chain': ['search', 'view', 'add', 'purchase'], 'weight': 0.8},
}
EVENT_CHAIN_NAMES = EVENT_CHAINS.keys()

def choose_event_chain(event_dict, conversion):
    chain_choice = None
    if not conversion:
        chain_choice = choices([value['chain'] for k, value in event_dict.items() if 'conversion' not in k],
                      [value['weight'] for k, value in event_dict.items()  if 'conversion' not in k],
                      k=1)[0]
    else:
        chain_choice = choices([value['chain'] for k, value in event_dict.items()  if 'conversion' in k],
                      [value['weight'] for k, value in event_dict.items()  if 'conversion' in k],
                      k=1)[0]
    yield chain_choice

## Generation Workflow
1. Did we convert?
    1. YES!
        1. How many touch points?
        2. What were our touch points leading up to the conversion? Choose [Search, NView]
        3. Generate conversion touch point.
    2. NO :(
        1. What type of touch point? Choose [Search, NView, Abandon]

In [364]:
if not is_conversion():
    for chain in choose_event_chain(EVENT_CHAINS, False):
        for event in chain:
            
else:
    for chain in choose_event_chain(EVENT_CHAINS, True):
        print(item)


['search', 'view']
